In [23]:
!pip install pdf2image
!pip install layoutparser torchvision && pip install "git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"
!pip install opencv-python
!apt-get install -y poppler-utils
!apt install tesseract-ocr
!pip install pytesseract
!pip install fpdf
!pip install layoutparser # Install the base layoutparser library with
!pip install layoutparser[layoutmodels] # Install DL layout model toolkit
!pip install layoutparser[ocr] # Install OCR toolkit!

  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git 'C:\Users\Huan Bui\AppData\Local\Temp\pip-install-vsob2cjv\detectron2_a8692511160c4574afa93a2f655b4122'
  Running command git checkout -q 82a57ce0b70057685962b352535147d9a8118578
  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [427 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\detectron2
  copying detectron2\__init__.py -> build\lib.win-amd64-cpython-310\detectron2
  creating build\lib.win-amd64-cpython-310\tools
  copying tools\analyze_model.py -> build\lib.win-amd64-cpython-310\tools
  copying tools\benchmark.py -> build\lib.win-amd64-cpython-310\tools
  copying tools\convert-torchvision-to-d2.py -> build\lib.win-amd64-cpython-310\tools
  copying tools\lazyconfig_train_net.py -> buil


  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.5) to c:\users\huan bui\appdata\local\temp\pip-install-vsob2cjv\detectron2_a8692511160c4574afa93a2f655b4122
  Resolved https://github.com/facebookresearch/detectron2.git to commit 82a57ce0b70057685962b352535147d9a8118578
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for detectron2
Failed to build detectron2
  Running setup.py install for detectron2: started
  Running setup.py install for detectron2: finished with status 'error'


'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


**Convert PDF files to images**

In [22]:
import os
from pdf2image import convert_from_path

directory = "data"
image_storage = "image_output"

# Ensure the output directory exists
if not os.path.exists(image_storage):
    os.makedirs(image_storage)

def pdf_to_jpeg(pdf_path, output_folder):
    images = convert_from_path(pdf_path)
    base_filename = os.path.basename(pdf_path).replace('.pdf', '')
    for i, image in enumerate(images):
        image.save(f'{output_folder}/{base_filename}_page_{i}.jpg', 'JPEG')

# Convert all PDFs in the directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(directory, filename)
        pdf_to_jpeg(pdf_path, image_storage)


PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

**Detect and extract tables as JSON**

In [ ]:
import os
import json
import cv2
import layoutparser as lp
import pytesseract

# Define the directories
directory = "data"
image_storage = "image_output"
csv_output = "csv_output"

# Ensure the output directory exists
if not os.path.exists(csv_output):
    os.makedirs(csv_output)

# Load the model
model = lp.Detectron2LayoutModel(
    'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"}
)

def ocr_table(image, table):
    x1, y1, x2, y2 = map(int, table.coordinates)
    cropped_table = image[y1:y2, x1:x2]
    table_text = pytesseract.image_to_string(cropped_table)
    return table_text.strip()

# Extract tables and save as JSON
for image_file in os.listdir(image_storage):
    image_path = os.path.join(image_storage, image_file)
    image = cv2.imread(image_path)
    
    layout = model.detect(image)
    tables = [b for b in layout if b.type == 'Table']
    
    tables_content = []
    for table in tables:
        table_data = {
            "coordinates": list(map(int, table.coordinates)),
            "type": table.type,
            "content": ocr_table(image, table)
        }
        tables_content.append(table_data)
    
    # Save with .json extension
    json_filename = os.path.splitext(image_file)[0] + ".json"
    with open(f"{csv_output}/{json_filename}", 'w', encoding="utf-8") as f:
        json.dump(tables_content, f, ensure_ascii=False, indent=4)


**Export text from the images, excluding tables and figures**

In [ ]:
import cv2
import layoutparser as lp
import pytesseract
from fpdf import FPDF

# Define the directories
directory = "data"
image_storage = "image_output"
pdf_output = "pdf_output"

# Ensure the output directory exists
if not os.path.exists(pdf_output):
    os.makedirs(pdf_output)

# Load the model (if not already loaded)
model = lp.Detectron2LayoutModel(
    'lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"}
)

def extract_text_from_image(image_path, excluding_regions=[]):
    image = cv2.imread(image_path)
    for region in excluding_regions:
        x1, y1, x2, y2 = map(int, region.coordinates)
        image[y1:y2, x1:x2] = 255  # Whiten the region in the image

    text = pytesseract.image_to_string(image)
    # Replace problematic characters
    text = text.replace("\u201c", '"').replace("\u201d", '"')
    return text

# Extract text from images and save as PDF (excluding tables and figures)
for image_file in os.listdir(image_storage):
    image_path = os.path.join(image_storage, image_file)
    layout = model.detect(image)

    non_text_regions = [b for b in layout if b.type in ['Table', 'Figure']]
    text = extract_text_from_image(image_path, excluding_regions=non_text_regions)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.multi_cell(0, 10, text.encode('latin-1', 'replace').decode('latin-1'))
    pdf.output(f"{pdf_output}/{image_file}.pdf")
